# SQL: JoinQueries

## Setup

We are now installing the necessary packages to interact with the MySQL database and issue SQL queries using the notebook.

In [0]:
!sudo apt-get install python3-mysqldb
!sudo pip3 install -U sqlalchemy sql_magic

In [3]:
%reload_ext sql_magic

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [0]:
from sqlalchemy import create_engine

conn_string = 'mysql://{user}:{password}@{host}/?charset=utf8'.format(
    host='db.ipeirotis.org',
    user='student',
    password='dwdstudent2015',
    encoding='utf-8')
engine = create_engine(conn_string)

In [0]:
%config SQL.conn_name = 'engine'

## Poor man's joins: Find the genres that Steven Spielberg typically directs

Use the table directors_genres  and find all genres of films and the corresponding probabilities for the director ID that corresponds to Steven Spielberg. Sort the results by probability. 


### IMDb

In [7]:
%%read_sql
USE imdb

Query started at 08:41:04 PM UTC; Query executed in 0.03 m

#### Find the entry for Steven Spielberg to find his id

In [8]:
%%read_sql
SELECT *
FROM directors 
WHERE first_name = 'Steven' AND last_name = 'Spielberg'

Query started at 08:41:27 PM UTC; Query executed in 0.01 m

,id,first_name,last_name
0,75380,Steven,Spielberg


#### Query the `directors_genres` table for Spielberg movies (pick director id value from previous query)

In [11]:
%%read_sql
SELECT *
FROM directors_genres
WHERE director_id = 75380
ORDER BY prob DESC

Query started at 08:42:23 PM UTC; Query executed in 0.01 m

,director_id,genre,prob
0,75380,Drama,0.435897
1,75380,Thriller,0.282051
2,75380,Adventure,0.282051
3,75380,Sci-Fi,0.256410
4,75380,Action,0.230769
5,75380,Horror,0.179487
6,75380,Short,0.128205
7,75380,Fantasy,0.102564
8,75380,War,0.102564
9,75380,Crime,0.102564


## Simple Join Queries

### IMDb

In [12]:
%%read_sql
USE imdb

Query started at 08:43:39 PM UTC; Query executed in 0.01 m

#### List all the movies and their genres

In [16]:
%%read_sql
SELECT *
FROM movies M 
  INNER JOIN movies_genres G ON M.id = G.movie_id


Query started at 08:45:58 PM UTC; Query executed in 0.05 m

,id,name,year,rank,movie_id,genre
0,1,"#7 Train: An Immigrant Journey, The",2000,NaN,1,Short
1,1,"#7 Train: An Immigrant Journey, The",2000,NaN,1,Documentary
2,2,$,1971,6.4,2,Crime
3,2,$,1971,6.4,2,Comedy
4,5,"$1,000 Reward",1923,NaN,5,Western
...,...,...,...,...,...,...
395445,378612,nz de mihlarim,1965,NaN,378612,Adventure
395446,378612,nz de mihlarim,1965,NaN,378612,Drama
395447,378613,egar a gerist,1998,NaN,378613,Comedy
395448,378613,egar a gerist,1998,NaN,378613,Drama


#### List the movie genres for the movies directed by Steven Spielberg and sort them in decreasing order of their probability (use the director_genres table)



In [18]:
%%read_sql
SELECT *
FROM directors D
  INNER JOIN directors_genres G ON G.director_id = D.id
WHERE D.first_name = 'Steven' AND D.last_name = 'Spielberg'
ORDER BY G.prob DESC

Query started at 08:47:12 PM UTC; Query executed in 0.01 m

,id,first_name,last_name,director_id,genre,prob
0,75380,Steven,Spielberg,75380,Drama,0.435897
1,75380,Steven,Spielberg,75380,Thriller,0.282051
2,75380,Steven,Spielberg,75380,Adventure,0.282051
3,75380,Steven,Spielberg,75380,Sci-Fi,0.256410
4,75380,Steven,Spielberg,75380,Action,0.230769
5,75380,Steven,Spielberg,75380,Horror,0.179487
6,75380,Steven,Spielberg,75380,Short,0.128205
7,75380,Steven,Spielberg,75380,War,0.102564
8,75380,Steven,Spielberg,75380,Fantasy,0.102564
9,75380,Steven,Spielberg,75380,Crime,0.102564


In [19]:
%%read_sql
SELECT G.genre, G.prob
FROM directors D
  INNER JOIN directors_genres G ON G.director_id = D.id
WHERE D.first_name = 'Steven' AND D.last_name = 'Spielberg'
ORDER BY G.prob DESC

Query started at 08:47:44 PM UTC; Query executed in 0.01 m

,genre,prob
0,Drama,0.435897
1,Thriller,0.282051
2,Adventure,0.282051
3,Sci-Fi,0.256410
4,Action,0.230769
5,Horror,0.179487
6,Short,0.128205
7,Fantasy,0.102564
8,War,0.102564
9,Crime,0.102564


#### Fetch all info for the directors with last names Scorsese, Polanski, and Spielberg. Use the OR for your Boolean query.


In [0]:
%%read_sql
SELECT *
FROM directors
WHERE last_name = 'Scorsese' OR last_name = 'Polanski' Or last_name = 'Spielberg'

Query started at 07:30:19 PM UTC; Query executed in 0.00 m

,id,first_name,last_name
0,63095,Roman,Polanski
1,71645,Martin,Scorsese
2,75380,Steven,Spielberg


#### Fetch all info for the directors Quentin Tarantino, Stanley Kubrick, and Orson Welles.

In [0]:
%%read_sql
SELECT *
FROM directors
WHERE (first_name = 'Quentin' AND last_name = 'Tarantino') OR 
      (first_name = 'Stanley' AND last_name = 'Kubrick') OR 
      (first_name = 'Orson' AND last_name = 'Welles')

Query started at 07:30:19 PM UTC; Query executed in 0.00 m

,id,first_name,last_name
0,43095,Stanley,Kubrick
1,78273,Quentin,Tarantino
2,84880,Orson,Welles


## `WHERE`: Inequality Queries

### IMDb

In [0]:
%%read_sql
USE imdb

Query started at 07:30:19 PM UTC; Query executed in 0.00 m

#### Find all information about movies that were released before 1895 (excl)

In [0]:
%%read_sql
SELECT *
FROM movies
WHERE year<1895

Query started at 07:30:19 PM UTC; Query executed in 0.00 m

,id,name,year,rank
0,13261,"Amateur Gymnast, No. 2",1894,NaN
1,17623,Annabelle Butterfly Dance,1894,5.3
2,17627,Annabelle Serpentine Dance,1894,6.1
3,17628,Annabelle Sun Dance,1894,NaN
4,17770,Annie Oakley,1894,4.9
...,...,...,...,...
76,363640,Whirlwind Gun Spinning,1894,NaN
77,364791,"Widder, The",1894,NaN
78,368839,Wrestling,1892,2.0
79,368852,"Wrestling Dog, The",1894,NaN


#### Find all information about movies released between 1895 and 1898 (excl) 

Try both using Boolean operators and using the BETWEEN operator

In [0]:
%%read_sql
SELECT *
FROM movies
WHERE year>1895 AND year<1898

Query started at 07:30:20 PM UTC; Query executed in 0.00 m

,id,name,year,rank
0,56,"'Amphitrite', The",1897,NaN
1,70,"'Brennus', Le",1897,NaN
2,247,"'Standard' Picture Animated, A",1897,NaN
3,1076,"13th Infantry, U.S. Army Marching Through Sall...",1897,NaN
4,1077,"13th Infantry, U.S. Army, in Heavy Marching Or...",1897,NaN
...,...,...,...,...
1093,372727,"Young Rivals, The",1897,NaN
1094,375815,"Zoo van Antwerpen, De",1897,NaN
1095,376589,"Z Pereira na Romaria de Santo Tirso, O",1896,NaN
1096,377666,"cole des gendres, L'",1897,NaN


In [0]:
%%read_sql
SELECT *
FROM movies
WHERE year BETWEEN 1896 AND 1897 -- notice that BETWEEN is inclusive on both sides

Query started at 07:30:20 PM UTC; Query executed in 0.00 m

,id,name,year,rank
0,56,"'Amphitrite', The",1897,NaN
1,70,"'Brennus', Le",1897,NaN
2,247,"'Standard' Picture Animated, A",1897,NaN
3,1076,"13th Infantry, U.S. Army Marching Through Sall...",1897,NaN
4,1077,"13th Infantry, U.S. Army, in Heavy Marching Or...",1897,NaN
...,...,...,...,...
1093,372727,"Young Rivals, The",1897,NaN
1094,375815,"Zoo van Antwerpen, De",1897,NaN
1095,376589,"Z Pereira na Romaria de Santo Tirso, O",1896,NaN
1096,377666,"cole des gendres, L'",1897,NaN


#### Find all information about movies that were released before 1895 and after 2006 (inclusive)

In [0]:
%%read_sql
SELECT *
FROM movies
WHERE year<=1895 OR year>=2006

Query started at 07:30:20 PM UTC; Query executed in 0.00 m

,id,name,year,rank
0,899,10th & Wolf,2006,NaN
1,1799,2176,2006,NaN
2,2238,300,2006,NaN
3,9086,Airborn,2006,NaN
4,9495,Akrobatisches Potpourri,1895,5.2
...,...,...,...,...
351,369459,X-Men 3,2006,NaN
352,372568,Young Griffo v. Battling Charles Barnett,1895,NaN
353,374889,Zhmurki,2006,NaN
354,400649,"""Pacific War, The""",2006,NaN


## The `IN` operator

### IMDb

In [0]:
%%read_sql
USE imdb

Query started at 07:30:20 PM UTC; Query executed in 0.00 m

#### Fetch all info for the directors with last names Scorsese, Polanski, and Spielberg. Use `IN` for your Boolean query.

In [0]:
%%read_sql
SELECT * 
FROM directors
WHERE last_name IN ( 'Scorsese', 'Spielberg', 'Polanski' );

Query started at 07:30:20 PM UTC; Query executed in 0.00 m

,id,first_name,last_name
0,63095,Roman,Polanski
1,71645,Martin,Scorsese
2,75380,Steven,Spielberg


#### Fetch all info for the directors Quentin Tarantino, Stanley Kubrick, and Orson Welles. Use `IN` for your Boolean query.


In [0]:
%%read_sql
SELECT * FROM directors
WHERE (first_name, last_name) IN (
            ('Quentin', 'Tarantino'), 
            ('Stanley', 'Kubrick'), 
            ('Orson', 'Welles') 
)


Query started at 07:30:20 PM UTC; Query executed in 0.00 m

,id,first_name,last_name
0,43095,Stanley,Kubrick
1,78273,Quentin,Tarantino
2,84880,Orson,Welles


## The `LIKE` operator for approximate queries

### IMDb

In [0]:
%%read_sql
USE imdb

Query started at 07:30:21 PM UTC; Query executed in 0.00 m

#### Find the entry for Alfred Hitchcock

Hint: Use an approximation for his first name


In [0]:
%%read_sql
SELECT *
FROM directors
WHERE last_name = 'Hitchcock' AND first_name LIKE 'A%%' -- The double %% is only necessary when writing SQL 
                                                        -- within Jupyter notebooks. 

Query started at 07:30:21 PM UTC; Query executed in 0.00 m

,id,first_name,last_name
0,34658,Alfred (I),Hitchcock


#### Find the Godfather movies, released in 1972, 1974, and 1990

In [0]:
%%read_sql
SELECT *
FROM movies
WHERE name LIKE 'Godfather%%'  AND -- The double %% is only necessary when writing SQL 
      year IN (1972, 1974, 1990)   -- within Jupyter notebooks. 

Query started at 07:30:21 PM UTC; Query executed in 0.00 m

,id,name,year,rank
0,130128,"Godfather, The",1972,9.0
1,130129,"Godfather: Part II, The",1974,8.9
2,130130,"Godfather: Part III, The",1990,7.3
